<a href="https://colab.research.google.com/github/juanCarlosEstrellaC/ticTacToe/blob/metodos-para-3-en-raya/ticTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import math

# Estado inicial= tablero vacío (np.nan, ... np.nan) + ficha de salida del jugador.
class TresEnRaya():
  def __init__(self):
    # Estado inicial = tablero vacío:
    self.estado = (np.nan, np.nan, np.nan,
                   np.nan, np.nan, np.nan,
                   np.nan, np.nan, np.nan)  #np.nan = null

  # Dado un estado, devuelve el los índices sin ocupar, ie, función movimientos disponibles.
  def f_sucesor(self, estado):
    sucesores = []
    for i in range(9):
      if np.isnan(estado[i]):
        sucesores.append(i)

    return sucesores


  def imprimir(self, estado):
    estado = list(estado)
    for i in range(9):
      if estado[i] == 1:
        estado[i] = "X"
      elif estado[i] == -1:
        estado[i] = "O"
      else:
        estado[i] = " "

    print("-------------")
    for i in [0,3,6]:
      print("| "+str(estado[i])+" | "+str(estado[i+1])+" | "+ str(estado[i+2])+" | \t")
      print("-------------")


  def aplicar_movimiento(self, estado_inicial, movimiento, jugador):
    estado = list(estado_inicial)
    estado[movimiento] = jugador
    return tuple(estado)


  def es_estado_ganador(self, estado, jugador): # Devuelve True si el humano o la máquina ganó. 
    return True if self.es_estado_final(estado, jugador) in [-1, 1] else False


  def es_estado_final(self, estado, jugador):       # Dado un estado, verifica si es estado final, y retorna:
    # Si es estado terminal -> devuelve jugador ganador (1 o -1) o 0 si empataron
    # Si NO es estado terminal -> devuelve nan

    # Verificamos si hay un ganador, de haberlo me lo devuelve:
    # Lineas Verticales:
    for i in range(3):   #pilas, que range(2) = [0,1], por eso el range(3)
      if estado[i] == jugador and estado[i+3] == jugador and estado[i+6] == jugador:
        return jugador

    # Lineas Horizontales:
    for i in [0,3,6]:    # Para mi es así "for i in [0,3,6]:"
      if estado[i] == jugador and estado[i+1] == jugador and estado[i+2] == jugador:
        return jugador

    # Lineas Diagonales:
    if estado[0] == jugador and estado[4] == jugador and estado[8] == jugador:
      return jugador
    if estado[2] == jugador and estado[4] == jugador and estado[6] == jugador:
      return jugador

    # Si no hay un ganador aún:
    completo = True
    # Verificamos si Tablero está completo. De ser cierto, se produjo un empate.
    for i in estado:
      if np.isnan(i):
        completo = False
        break
    
    if completo: # Este caso sería el empate.
      return 0

    # Ya que NO es estado terminal -> devuelve nan
    return np.nan 


In [6]:
# Existen 2 jugadores: X y O. 
# Minimax es un algoritmo recursivo, por lo que es importante ponerle una profundidad limitada
class jugador():

  def decide_jugada(self, estado, jugador): # Decision-minimax
    raise NotImplementedError("El método debe ser implementado") #Es una excepción.

class jugador_Maquina(jugador): #Definir herencia ej: perro(animales)
  
  def f_score(self,e,j): # e = estado  y j = jugador. Esta es una función que devuelve el número de líneas ganadoras para usarlo en f_evaluacion y en consecuencia en el minimax
    lineas = 0
    # diagonales.
    if(np.isnan(e[0]) or e[0]==j) and (np.isnan(e[4]) or e[4]==j)and (np.isnan(e[8]) or e[8]==j):
      lineas+= 1
    if(np.isnan(e[2]) or e[2]==j) and (np.isnan(e[4]) or e[4]==j)and (np.isnan(e[6]) or e[6]==j):
      lineas+= 1

    for i in range(3):
      # columnas:
     if(np.isnan(e[i]) or e[i]==j) and (np.isnan(e[i+3]) or e[i+3]==j)and (np.isnan(e[i+6]) or e[i+6]==j):
      lineas+= 1

      # filas:
     if(np.isnan(e[3*i]) or e[3*i]==j) and (np.isnan(e[3*i+3]) or e[3*i+3]==j)and (np.isnan(e[3*i+6]) or e[3*i+6]==j):
      lineas+= 1
    return lineas  


  def f_evaluacion(self, estado, juego, jugador):
    final = juego.es_estado_final(estado, jugador) # "final" puede ser -1, 1 o nan
    if not np.isnan(final): # si final NO es nan
      if juego.es_estado_ganador(estado, jugador):
        if final == 1:
          return math.inf
        else:
          return -math.inf
      else:
        return 0
    else:
      return self.f_score(estado, 1) - self.f_score(estado, -1)  # Diferencia entre líneas ganadoras para X y O


  def f_maximizador(self, estado, juego, jugador, movimiento, profundidad):
    max_valor = - math.inf
    for mov in movimiento:
      sucesor = juego.aplicar_movimiento(estado, mov, jugador)
      valor_actual = self.minimax(estado, juego, jugador, 'MIN', profundidad)
      if valor_actual > max_valor:
        max_valor = valor_actual
    return max_valor

  def f_minimizador(self, estado, juego, jugador, movimiento, profundidad):
    min_valor =  math.inf
    for mov in movimiento:
      sucesor = juego.aplicar_movimiento(estado, mov, jugador)
      valor_actual = self.minimax(estado, juego, jugador, 'MAX', profundidad)
      if valor_actual > min_valor:
        min_valor = valor_actual
    return min_valor


  # Este método devuelve el mejor movimiento para X y el que es peor para O.
  def minimax(self, estado, juego, jugador, turno, profundidad): # valor-minimax() del pdf
    valor_evaluacion = self.f_evaluacion(estado)
    final = juego.es_estado_final(estado, jugador) # "final" puede ser -1, 1 o nan
    
    if not np.isnan(final): # si final NO es nan
      return valor_evaluacion
    if profundidad == 0:
      return valor_evaluacion
    
    movimientos = juego.f_sucesor(estado)
    if not movimientos:
      return valor_evaluacion

    if turno == 'MAX':
      valor_evaluacion = self.f_maximizador(estado, juego, jugador, movimientos, profundidad-1)
    else:
      valor_evaluacion = self.f_minimizador(estado, juego, jugador, movimientos, profundidad-1)
    return valor_evaluacion
  
  # Método para que la Máquina decida qué jugada (índice del 0 al 8) hacer
  def decide_jugada(self, estado, juego, jugador): # juego= instancia de la clase TresEn Raya []
    valor_max = math.inf
    decision = np.nan
    profundidad = 10   # Es 
    movimientos = juego.f_sucesor(estado) # movimientos = sucesor del estado
    for mov in movimientos:               # for i in sucesor
      nuevo_estado = juego.aplicar_movimiento(estado, mov, jugador) # aplica el movimiento al estado y lo guardo en uno nuevo
      valor_actual = self.minimax(nuevo_estado, juego, jugador, 'MAX', profundidad) # evalúa qué tan buena es el mov o índice.
      if valor_actual > valor_max:
        valor_max = valor_actual
        decision = mov
    return decision


class jugador_Humano(jugador):  
  def decide_jugada(self, estado, jugador):
    pass


In [7]:
# jugador:  1 = X (máquina) y  -1 = O (humano)
# diferentes estados para probar:
estado1 = (np.nan, np.nan, np.nan,
           np.nan, np.nan, np.nan,
           np.nan, np.nan, np.nan)

estado2 = range(9)

estado3 = (1,      1,      1,  
          np.nan, -1, np.nan,
          np.nan, -1, np.nan)

estado4 = (1,1,1,1,1,1,1,1,1)

estado5 = (1,      -1,  np.nan,  
           1,       1,     1,
           np.nan, -1, np.nan)

# Probando métodos
j = TresEnRaya()   
ganador = j.es_estado_final(estado3, 1)
print(ganador)

sucesor = j.f_sucesor(estado3) 
print(sucesor)

nuevo_estado = j.aplicar_movimiento(estado3, sucesor.pop(), -1)
print(nuevo_estado)

print()
j.imprimir(estado5)


1
[3, 5, 6, 8]
(1, 1, 1, nan, -1, nan, nan, -1, -1)

-------------
| X | O |   | 	
-------------
| X | X | X | 	
-------------
|   | O |   | 	
-------------


In [8]:
juego = TresEnRaya()
max = jugador_Maquina()
min = jugador_Humano()

for turno in range(10):
  max.decide_jugada()

In [9]:
"""def f_sucesorMio(self, estado):
    sucesores = [] 
    print(estado)
    for i in estado:
      if np.isnan(i):
        print("Es nulo "+str(i))
        sucesores.append(estado.index(i))
        print(estado.index(i))
    return sucesores

  def f_sucesor2(self, estado):
    sucesores = [] 
    for i,e in enumerate(estado):
      print(i)
      if np.isnan(i):
        sucesores.append(e)
        print(estado.index(i))
    return sucesores
  """

"""def aplicar_movimientoMio(self, estado, movimiento, jugador):
    suc = list(self.f_sucesor(estado))
    estado_list = list(estado)
    if movimiento in suc:
      estado_list[movimiento] = jugador
      return estado_list
    else:
      print("Movimiento inválido")
"""

'def aplicar_movimientoMio(self, estado, movimiento, jugador):\n    suc = list(self.f_sucesor(estado))\n    estado_list = list(estado)\n    if movimiento in suc:\n      estado_list[movimiento] = jugador\n      return estado_list\n    else:\n      print("Movimiento inválido")\n'